# Evaluate Policies and Model Checker on Urban Scenarios

**Supported Policies**
    - baseline
    - masked-baseline
    - masked-RL

**Supported Belief Updaters**
    - previous_obs
    - tracker
    
**Supported Scenarios**
    - 1-1 : One car going straight + noisy sensor
    - 1-2 : One car going straight + occlusion
    - 2-1 : One car and one pedestrian interacting
    - 2-2 : One car turning left interacting with the ego vehicle
    - 3 : multiple cars and multiple pedestrian

In [42]:
using Revise
includet("../evaluation/helpers.jl")
includet("../src/render_helpers.jl")

In [43]:
rng = MersenneTwister(1);

In [44]:
pomdp, policy, updater, initialscene_scenario = load_policies_and_environment("masked-RL", "tracker", "3");

Evaluating masked-RL policy with tracker updater on scenario 3


## Visualize simulations

In [48]:
@time hist = run_sim(pomdp, policy, updater, initialscene_scenario, rng);
@printf("Simulation: %i steps, %0.3f discounted reward ", n_steps(hist), undiscounted_reward(hist))

 37.579676 seconds (101.66 M allocations: 9.093 GiB, 29.32% gc time)
Simulation: 153 steps, 1.000 discounted reward 

In [49]:
animate_history(hist, pomdp,
#                 obs_overlays = o -> [GaussianSensorOverlay(sensor=pomdp.sensor, o=[veh for veh in obs_to_scene(pomdp, o) if veh.id != EGO_ID], color=MONOKAI["color2"])],
#                 belief_overlays = b -> [MultipleAgentsBeliefOverlay(b)],
                action_overlays = a -> [TextOverlay(text = ["Acc: $(a.acc) m/s^2"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 8.), incameraframe=true)],
#                 ainfo_overlays = ai -> [MultipleInterpolatedBeliefsOverlay(ai[5], pomdp, mdp, pedcar_pomdp, ai[end]),
#                                         TextOverlay(text = ["Probabilities: $([round(p, digits=4) for p in ai[2]])"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3., 12.), incameraframe=true),
#                                         TextOverlay(text = ["Available Actions: $([a.acc for a in ai[1]])"], font_size=20,pos=VecE2(pomdp.env.params.x_min + 3.,10.), incameraframe=true)],                
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                cam =  StaticCamera(VecE2(0., -8.), 12.0),
                speed_factor=2)

Frames{MIME{Symbol("image/png")}}("/tmp/tmp4NEsn9", 0x0000000000000099, 20.0, nothing)

## Collision Analysis

In [ ]:
hist = find_collision(pomdp, policy, updater, initialscene_scenario, rng, n_ep=1000);

Progress:   9%|████                                     |  ETA: 11:11:15

In [ ]:

@time rewards, steps, violations = evaluation_loop(pomdp, policy, updater,
                set_static_spec!, initialscene_scenario,
                n_ep=N_EPISODES, max_steps=MAX_STEPS, rng=rng)
print_summary(rewards, steps, violations)

In [45]:
pomdp.ped_birth

0.1

In [46]:
pomdp.car_birth

0.1